In [ ]:
from bs4 import BeautifulSoup as bs
import requests
import re
from splinter import Browser
import pandas as pd
import itertools

In [ ]:
#Define Executable Path for ChromeDriver to use for web scraping

executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
# Define base endpoint URL. Other url's will be built off this endpoint.

baseball_reference_main = 'https://www.baseball-reference.com'
nats_schedule_endpoint = '/teams/WSN/2021-schedule-scores.shtml'
nats_schedule_page = baseball_reference_main + nats_schedule_endpoint
browser.visit(nats_schedule_page)

In [ ]:
#Use Beautiful Soup HTML.parser to extract relvant details about news articles

schedule_html = browser.html
schedule_soup = bs(schedule_html, 'html.parser')
schedule_soup

In [ ]:
# Extract list of games

game_listings = schedule_soup.find('table', class_="sortable stats_table now_sortable", id="team_schedule").find('tbody').find_all('tr',class_=lambda x: x != 'thead')
game_listings

In [ ]:
test = game_listings[31]
test

In [ ]:
boxscore_braves = test.find_all('td')[1]
boxscore_braves_endpoint = boxscore_braves.a['href']
boxscore_braves_endpoint

In [ ]:
boxscore_url = baseball_reference_main + boxscore_braves_endpoint
browser.visit(boxscore_url)
box_html = browser.html
box_soup = bs(box_html, 'html.parser')

In [ ]:
box_soup

In [ ]:
nats_pitching = box_soup.find('table', id="WashingtonNationalspitching")
nats_pitching

In [ ]:
nats_pitch_headers = box_soup.find('div', id="div_WashingtonNationalspitching").find('thead').find('tr').find_all('th')
nats_pitching_headers = []

for header in nats_pitch_headers:
    one_header = header.text
    nats_pitching_headers.append(one_header)
    
nats_pitching_headers.append('W')
nats_pitching_headers.append('L')
nats_pitching_headers.append('H')
nats_pitching_headers.append('S')
nats_pitching_headers.append('BS')

nats_pitching_headers

In [ ]:
nats_hs_location = box_soup.find('div', id="div_WashingtonNationalspitching").find('tbody')
names_positions_stats = nats_hs_location.find_all('tr')

All_pitchers_Info = []

for players in names_positions_stats:
    list_atts = []
    names_and_records = players.find('th')
    player_content_length = len(names_and_records.contents)  
    player_name = names_and_records.contents[0].text
    list_atts.append(player_name)
    print(player_name)
    print(player_content_length)
    stats_for_players = players.find_all('td')
    for stat in stats_for_players:
        each_stat = stat.text
        list_atts.append(each_stat)      
    if player_content_length == 2:
        player_decision = names_and_records.contents[1]
        print(player_decision.split())
        decision_split = player_decision.split()
        decisionsToCheck = ['W', 'L', 'H', 'S', 'BS']
        for decision in decisionsToCheck:
            if decision in decision_split:
                list_atts.append(1)
            else:
                list_atts.append(0)
    else:
        list_atts.extend([0] * 5)
    
    All_pitchers_Info.append(list_atts)

In [ ]:
All_pitchers_Info

In [ ]:
pitching_df = pd.DataFrame(All_pitchers_Info, columns = nats_pitching_headers)

In [ ]:
pitching_df

In [ ]:
pitching_df = pitching_df.replace("", 0)

In [ ]:
pitching_df

In [ ]:
pitching_df.dtypes

In [ ]:
pitching_df['Pitching'] = pitching_df['Pitching'].astype(str)
pitching_df['IP'] = pitching_df['IP'].astype(float)
pitching_df['H'] = pitching_df['H'].astype(int)
pitching_df['R'] = pitching_df['R'].astype(int)
pitching_df['ER'] = pitching_df['ER'].astype(int)
pitching_df['BB'] = pitching_df['BB'].astype(int)
pitching_df['SO'] = pitching_df['SO'].astype(int)
pitching_df['HR'] = pitching_df['HR'].astype(int)
pitching_df['ERA'] = pitching_df['ER'].astype(int)
pitching_df['BF'] = pitching_df['BF'].astype(int)
pitching_df['Pit'] = pitching_df['Pit'].astype(int)
pitching_df['Str'] = pitching_df['Str'].astype(int)
pitching_df['Ctct'] = pitching_df['Ctct'].astype(int)
pitching_df['StS'] = pitching_df['StS'].astype(int)
pitching_df['StL'] = pitching_df['StL'].astype(int)
pitching_df['GB'] = pitching_df['GB'].astype(int)
pitching_df['FB'] = pitching_df['FB'].astype(int)
pitching_df['LD'] = pitching_df['LD'].astype(int)
pitching_df['Unk'] = pitching_df['Unk'].astype(int)
pitching_df['GSc'] = pitching_df['GSc'].astype(int)
pitching_df['IR'] = pitching_df['IR'].astype(int)
pitching_df['IS'] = pitching_df['IS'].astype(int)
pitching_df['WPA'] = pitching_df['WPA'].astype(float)
pitching_df['aLI'] = pitching_df['aLI'].astype(float)
pitching_df['cWPA'] = pitching_df['cWPA'].astype(str)
pitching_df['acLI'] = pitching_df['acLI'].astype(float)
pitching_df['RE24'] = pitching_df['RE24'].astype(float)
pitching_df['W'] = pitching_df['W'].astype(int)
pitching_df['L'] = pitching_df['L'].astype(int)
pitching_df['H'] = pitching_df['H'].astype(int)
pitching_df['S'] = pitching_df['S'].astype(int)
pitching_df['BS'] = pitching_df['BS'].astype(int)

pitching_df.dtypes

In [ ]:
pitching_df